In [1]:
from google.colab import files
files.upload()

Saving cardekho_imputated.csv to cardekho_imputated.csv


{'cardekho_imputated.csv': b',car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price\n0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.7,796,46.3,5,120000\n1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.9,1197,82,5,550000\n2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17,1197,80,5,215000\n3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.1,5,226000\n4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000\n5,Maruti Wagon R,Maruti,Wagon R,8,35000,Individual,Petrol,Manual,18.9,998,67.1,5,350000\n6,Hyundai i10,Hyundai,i10,8,40000,Dealer,Petrol,Manual,20.36,1197,78.9,5,315000\n7,Maruti Wagon R,Maruti,Wagon R,3,17512,Dealer,Petrol,Manual,20.51,998,67.04,5,410000\n8,Hyundai Venue,Hyundai,Venue,2,20000,Individual,Petrol,Automatic,18.15,998,118.35,5,1050000\n12,Maruti Swift,Maruti,Swift,4,28321,Dealer,Petrol,Manual,16.6,1197,8

In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from mlxtend.classifier import StackingClassifier
from sklearn.metrics import r2_score, mean_squared_error

In [33]:
data = pd.read_csv('cardekho_imputated.csv')
data.head()

,Unnamed: 0,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15411 entries, 0 to 15410
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         15411 non-null  int64  
 1   car_name           15411 non-null  object 
 2   brand              15411 non-null  object 
 3   model              15411 non-null  object 
 4   vehicle_age        15411 non-null  int64  
 5   km_driven          15411 non-null  int64  
 6   seller_type        15411 non-null  object 
 7   fuel_type          15411 non-null  object 
 8   transmission_type  15411 non-null  object 
 9   mileage            15411 non-null  float64
 10  engine             15411 non-null  int64  
 11  max_power          15411 non-null  float64
 12  seats              15411 non-null  int64  
 13  selling_price      15411 non-null  int64  
dtypes: float64(2), int64(6), object(6)
memory usage: 1.6+ MB


In [35]:
data.describe()

,Unnamed: 0,vehicle_age,km_driven,mileage,engine,max_power,seats,selling_price
count,15411.000000,15411.000000,1.541100e+04,15411.000000,15411.000000,15411.000000,15411.000000,1.541100e+04
mean,9811.857699,6.036338,5.561648e+04,19.701151,1486.057751,100.588254,5.325482,7.749711e+05
std,5643.418542,3.013291,5.161855e+04,4.171265,521.106696,42.972979,0.807628,8.941284e+05
min,0.000000,0.000000,1.000000e+02,4.000000,793.000000,38.400000,0.000000,4.000000e+04
25%,4906.500000,4.000000,3.000000e+04,17.000000,1197.000000,74.000000,5.000000,3.850000e+05
50%,9872.000000,6.000000,5.000000e+04,19.670000,1248.000000,88.500000,5.000000,5.560000e+05
75%,14668.500000,8.000000,7.000000e+04,22.700000,1582.000000,117.300000,5.000000,8.250000e+05
max,19543.000000,29.000000,3.800000e+06,33.540000,6592.000000,626.000000,9.000000,3.950000e+07


In [36]:
data.describe(include='object')

,car_name,brand,model,seller_type,fuel_type,transmission_type
count,15411,15411,15411,15411,15411,15411
unique,121,32,120,3,5,2
top,Hyundai i20,Maruti,i20,Dealer,Petrol,Manual
freq,906,4992,906,9539,7643,12225


In [37]:
data.columns

Index(['Unnamed: 0', 'car_name', 'brand', 'model', 'vehicle_age', 'km_driven',
       'seller_type', 'fuel_type', 'transmission_type', 'mileage', 'engine',
       'max_power', 'seats', 'selling_price'],
      dtype='object')

In [38]:
data.drop('Unnamed: 0',axis = 1, inplace = True)
data.head()

,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [39]:
print(data.car_name.unique())
print(data.brand.unique())
print(data.model.unique())
print(data.seller_type.unique())
print(data.fuel_type.unique())
print(data.transmission_type.unique())

['Maruti Alto' 'Hyundai Grand' 'Hyundai i20' 'Ford Ecosport'
 'Maruti Wagon R' 'Hyundai i10' 'Hyundai Venue' 'Maruti Swift'
 'Hyundai Verna' 'Renault Duster' 'Mini Cooper' 'Maruti Ciaz'
 'Mercedes-Benz C-Class' 'Toyota Innova' 'Maruti Baleno'
 'Maruti Swift Dzire' 'Volkswagen Vento' 'Hyundai Creta' 'Honda City'
 'Mahindra Bolero' 'Toyota Fortuner' 'Renault KWID' 'Honda Amaze'
 'Hyundai Santro' 'Mahindra XUV500' 'Mahindra KUV100' 'Maruti Ignis'
 'Datsun RediGO' 'Mahindra Scorpio' 'Mahindra Marazzo' 'Ford Aspire'
 'Ford Figo' 'Maruti Vitara' 'Tata Tiago' 'Volkswagen Polo' 'Kia Seltos'
 'Maruti Celerio' 'Datsun GO' 'BMW 5' 'Honda CR-V' 'Ford Endeavour'
 'Mahindra KUV' 'Honda Jazz' 'BMW 3' 'Audi A4' 'Tata Tigor'
 'Maruti Ertiga' 'Tata Safari' 'Mahindra Thar' 'Tata Hexa'
 'Land Rover Rover' 'Maruti Eeco' 'Audi A6' 'Mercedes-Benz E-Class'
 'Audi Q7' 'BMW Z4' 'BMW 6' 'Jaguar XF' 'BMW X5' 'MG Hector' 'Honda Civic'
 'Isuzu D-Max' 'Porsche Cayenne' 'BMW X1' 'Skoda Rapid' 'Ford Freestyle'
 'Skoda

In [40]:
encoder = LabelEncoder()
for i in data.columns:
  if data[i].dtype == 'object':
    data[i] = encoder.fit_transform(data[i])
data.head()

,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,65,18,7,9,120000,1,4,1,19.70,796,46.30,5,120000
1,34,8,54,5,20000,1,4,1,18.90,1197,82.00,5,550000
2,40,8,118,11,60000,1,4,1,17.00,1197,80.00,5,215000
3,65,18,7,9,37000,1,4,1,20.92,998,67.10,5,226000
4,20,6,38,6,30000,0,1,1,22.77,1498,98.59,5,570000


In [44]:
correlation_matrix = data.corr()
important_features = correlation_matrix['selling_price'].drop('selling_price').abs().sort_values(ascending=False)
print(important_features)

max_power            0.750236
engine               0.585844
transmission_type    0.459394
mileage              0.305549
vehicle_age          0.241851
fuel_type            0.217803
seller_type          0.137198
seats                0.115033
model                0.110702
km_driven            0.080030
car_name             0.056979
brand                0.049755
Name: selling_price, dtype: float64


In [45]:
X = data[important_features.head(9).index]
y = data['selling_price']

In [52]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
y = scaler.fit_transform(y.values.reshape(-1, 1))

In [53]:
X[:5]

array([[-1.26335238e+00, -1.32425883e+00,  5.10503180e-01,
        -2.75974621e-04,  9.83561835e-01,  1.00142330e+00,
         1.18996622e+00, -4.03022414e-01, -1.51971354e+00],
       [-4.32570824e-01, -5.54717742e-01,  5.10503180e-01,
        -1.92070558e-01, -3.43933310e-01,  1.00142330e+00,
         1.18996622e+00, -4.03022414e-01, -2.25693398e-01],
       [-4.79113208e-01, -5.54717742e-01,  5.10503180e-01,
        -6.47582693e-01,  1.64730941e+00,  1.00142330e+00,
         1.18996622e+00, -4.03022414e-01,  1.53637659e+00],
       [-7.79311584e-01, -9.36609701e-01,  5.10503180e-01,
         2.92210765e-01,  9.83561835e-01,  1.00142330e+00,
         1.18996622e+00, -4.03022414e-01, -1.51971354e+00],
       [-4.65017488e-02,  2.29178336e-02,  5.10503180e-01,
         7.35735739e-01, -1.20595237e-02, -9.67649274e-01,
        -7.68027526e-01, -4.03022414e-01, -6.66210895e-01]])

In [54]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [55]:
linear_model = LinearRegression()
linear_model.fit(x_train, y_train)

LinearRegression()

In [56]:
y_pred = linear_model.predict(x_test)

In [57]:
print(r2_score(y_test, y_pred))
print(mean_squared_error(y_test, y_pred))

0.6679554590609486
0.3126755252322949


In [59]:
decision_model = DecisionTreeRegressor()
decision_model.fit(x_train, y_train)

DecisionTreeRegressor()

In [60]:
y_pred = decision_model.predict(x_test)

In [61]:
print(r2_score(y_test, y_pred))
print(mean_squared_error(y_test, y_pred))

0.8919634031153592
0.1017345431419557


In [62]:
random_model = RandomForestRegressor()
random_model.fit(x_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor()

In [63]:
y_pred = random_model.predict(x_test)

In [65]:
print(r2_score(y_test, y_pred))
print(mean_squared_error(y_test, y_pred))

0.9294296179783225
0.06645382936297693


In [67]:
base_models = [
    ('Linear Regression', LinearRegression()),
    ('Decision Tree', DecisionTreeRegressor()),
    ('Random Forest', RandomForestRegressor())
]
meta_model = RandomForestRegressor()
stacking_model = StackingClassifier(classifiers=[model for _, model in base_models], meta_classifier=meta_model)
stacking_model.fit(x_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.12/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


StackingClassifier(classifiers=[LinearRegression(), DecisionTreeRegressor(),
                                RandomForestRegressor()],
                   meta_classifier=RandomForestRegressor())

In [68]:
y_pred = stacking_model.predict(x_test)

In [69]:
print(r2_score(y_test, y_pred))
print(mean_squared_error(y_test, y_pred))

0.9018802910056567
0.09239613293650757
